In [1]:
# Install extra-dependency
! pip -q install pycocotools

menpo 0.8.1 has requirement matplotlib<2.0,>=1.4, but you'll have matplotlib 2.2.3 which is incompatible.
menpo 0.8.1 has requirement pillow<5.0,>=3.0, but you'll have pillow 5.2.0 which is incompatible.
menpo 0.8.1 has requirement scipy<1.0,>=0.16, but you'll have scipy 1.1.0 which is incompatible.
You are using pip version 10.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import sys
sys.path.append("slim")

import functools
import json
import os
import tensorflow as tf

from object_detection import trainer
from object_detection.builders import dataset_builder
from object_detection.builders import graph_rewriter_builder
from object_detection.builders import model_builder
from object_detection.utils import config_util
from object_detection.utils import dataset_util

from support import get_train_config, get_eval_config

# For reproducibility
from tensorflow import set_random_seed
from numpy.random import seed
seed(2018)
set_random_seed(2018)

In [3]:
##################################################
# CONFIGURATION (Local, Multi-GPUs, Distributed) #
##################################################

MASTER = ''  # Name of the TensorFlow master to use
TASK = 0  # task id
NUM_CLONES = 1  # Number of clones to deploy per worker.

# Force clones to be deployed on CPU.  Note that even if 
# set to False (allowing ops to run on gpu), some ops may 
# still be run on the CPU if they have no GPU kernel.
CLONE_ON_CPU = False

WORKER_REPLICAS = 1  # Number of worker+trainer replicas.
PS_TASKS = 0  # Number of parameter server tasks. If None, does not use a parameter server.
TRAIN_DIR = 'trained_models/ssdlite_mobilenet_v2_coco_2018_05_09'  # Directory to save the checkpoints and training summaries.

# Path to a pipeline_pb2.TrainEvalPipelineConfig config
# file. If provided, other configs are ignored
PIPELINE_CONFING_PATH = 'models/ssdlite_mobilenet_v2_coco_2018_05_09/pipeline.config'
TRAIN_CONFIG_PATH = ''  # Path to a train_pb2.TrainConfig config file.
INPUT_CONFIG_PATH = ''  # Path to an input_reader_pb2.InputReader config file.
MODEL_CONFIG_PATH = ''  # Path to a model_pb2.DetectionModel config file.

# Get configuration for training
(create_input_dict_fn,
model_fn,
train_config,
master,
task,
worker_job_name,
ps_tasks,
worker_replicas,
is_chief,
graph_rewriter_fn) = get_train_config(TASK,
                                     PS_TASKS,
                                     TRAIN_DIR,
                                     PIPELINE_CONFING_PATH,
                                     TRAIN_CONFIG_PATH,
                                     MODEL_CONFIG_PATH,
                                     INPUT_CONFIG_PATH,
                                     WORKER_REPLICAS,
                                     MASTER)


######################################
# HOW TO EDIT TRAIN CONFIG FROM CODE #
######################################

# You can change the setting in this way,
# e.g. for batch size and num_steps
# train_config.batch_size = 4
# train_config.num_steps = 4000

print("Train configuration: \n", '-'*30, '\n', train_config)

Train configuration: 
 ------------------------------ 
 batch_size: 8
data_augmentation_options {
  random_horizontal_flip {
  }
}
data_augmentation_options {
  ssd_random_crop {
  }
}
optimizer {
  rms_prop_optimizer {
    learning_rate {
      exponential_decay_learning_rate {
        initial_learning_rate: 0.004000000189989805
        decay_steps: 10000
        decay_factor: 0.949999988079071
      }
    }
    momentum_optimizer_value: 0.8999999761581421
    decay: 0.8999999761581421
    epsilon: 1.0
  }
}
fine_tune_checkpoint: "/floyd/home/workspace-focas/models/ssdlite_mobilenet_v2_coco_2018_05_09/model.ckpt"
num_steps: 7400
fine_tune_checkpoint_type: "detection"



In [4]:
# Training
trainer.train(
  create_input_dict_fn,
  model_fn,
  train_config,
  master,
  task,
  NUM_CLONES,
  worker_replicas,
  CLONE_ON_CPU,
  ps_tasks,
  worker_job_name,
  is_chief,
  TRAIN_DIR,
  graph_hook_fn=graph_rewriter_fn)

Instructions for updating:
Please switch to tf.train.create_global_step
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
Instructions for updating:
Please switch to tf.train.MonitoredTrainingSession
INFO:tensorflow:Restoring parameters from /floyd/home/workspace-focas/models/ssdlite_mobilenet_v2_coco_2018_05_09/model.ckpt
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Starting Session.
INFO:tensorflow:Saving checkpoint to path trained_models/ssdlite_mobilenet_v2_coco_2018_05_09/model.ckpt
INFO:tensorflow:Starting Queues.
INFO:tensorflow:global_step/sec: 0
INFO:tensorflow:Recording summary at step 0.


## Evaluate

Now it's time to evaluate our trained model.

**Note**

Unfortunately a bug prevent the correct output of [IOU(Intersection Over Union) metric](https://en.wikipedia.org/wiki/Jaccard_index). We will fix it as soon as possible.

In [9]:
from object_detection import evaluator
from object_detection.utils import label_map_util

tf.reset_default_graph()

##############################
# CONFIGURATION (Evaluation) #
##############################

# Directory to write eval summaries to.
EVAL_DIR = 'trained_models/ssdlite_mobilenet_v2_coco_2018_05_09/eval' 

# If training data should be evaluated for this job.
EVAL_TRAINING_DATA = False

# Option to only run a single pass of evaluation. 
# Overrides the `max_evals` parameter in the provided config
RUN_ONCE = True

# Directory containing checkpoints to evaluate, typically
# set to `train_dir` used in the training job.
CHECKPOINT_DIR = TRAIN_DIR

EVAL_CONFIG_PATH = ''  # Path to a eval_pb2.TrainConfig config file.
EVAL_INPUT_CONFIG_PATH = ''  # Path to an input_reader_pb2.InputReader config file.
MODEL_CONFIG_PATH = ''  # Path to a model_pb2.DetectionModel config file.

# Get configuration for Evaluation
(create_input_dict_fn,
model_fn,
eval_config,
categories,
graph_rewriter_fn) = get_eval_config(EVAL_DIR,
                                    PIPELINE_CONFING_PATH,
                                    EVAL_CONFIG_PATH,
                                    MODEL_CONFIG_PATH,
                                    EVAL_INPUT_CONFIG_PATH,
                                    EVAL_TRAINING_DATA,
                                    RUN_ONCE)

######################################
# HOW TO EDIT EVAL CONFIG FROM CODE #
######################################

# You can change the setting in this way,
# e.g. for evaluate only once
# eval_config.max_evals = 1

print("Eval configuration: \n", '-'*30, '\n', eval_config)

Eval configuration: 
 ------------------------------ 
 num_examples: 1
max_evals: 1



In [10]:
# Evaluate
evaluator.evaluate(
    create_input_dict_fn,
    model_fn,
    eval_config,
    categories,
    CHECKPOINT_DIR,
    EVAL_DIR,
    graph_hook_fn=graph_rewriter_fn)

INFO:tensorflow:depth of additional conv before box predictor: 0


INFO:tensorflow:depth of additional conv before box predictor: 0


INFO:tensorflow:depth of additional conv before box predictor: 0


INFO:tensorflow:depth of additional conv before box predictor: 0


INFO:tensorflow:depth of additional conv before box predictor: 0


...

INFO:tensorflow:depth of additional conv before box predictor: 0


INFO:tensorflow:depth of additional conv before box predictor: 0


INFO:tensorflow:Restoring parameters from trained_models/ssdlite_mobilenet_v2_coco_2018_05_09/model.ckpt-7400


INFO:tensorflow:Restoring parameters from trained_models/ssdlite_mobilenet_v2_coco_2018_05_09/model.ckpt-7400


{'PascalBoxes_Precision/mAP@0.5IOU': 0.0,
 'PascalBoxes_PerformanceByCategory/AP@0.5IOU/foca': 0.0,
 'Losses/Loss/localization_loss': 2.9487154,
 'Losses/Loss/classification_loss': 15.074476}

## Exporting the model

In the last section of the notebook, we will export the model to run prediction on images converted into tensors (matrix representation for [height, width, color_channels]).

In [13]:
# Exporting the model for Evaluation
from google.protobuf import text_format
from object_detection import exporter
from object_detection.protos import pipeline_pb2

slim = tf.contrib.slim
tf.reset_default_graph()

config_override=''
input_shape=None
trained_checkpoint_prefix = os.path.join(TRAIN_DIR, 'model.ckpt-7400') # EDIT: model.ckpt-<iteration>
output_directory='trained_models/ssdlite_mobilenet_v2_coco_2018_05_09_exported'

# Type of input node. Can be one of :
# [`image_tensor`, `encoded_image_string_tensor`,`tf_example`]
input_type='image_tensor' 

pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.gfile.GFile(PIPELINE_CONFING_PATH, 'r') as f:
    text_format.Merge(f.read(), pipeline_config)
text_format.Merge(config_override, pipeline_config)
if input_shape:
    input_shape = [
        int(dim) if dim != '-1' else None
        for dim in input_shape.split(',')
    ]
else:
    input_shape = None
exporter.export_inference_graph(input_type, pipeline_config,
                                  trained_checkpoint_prefix,
                                  output_directory, input_shape)

INFO:tensorflow:depth of additional conv before box predictor: 0


INFO:tensorflow:depth of additional conv before box predictor: 0


INFO:tensorflow:depth of additional conv before box predictor: 0


INFO:tensorflow:depth of additional conv before box predictor: 0


INFO:tensorflow:depth of additional conv before box predictor: 0


...

INFO:tensorflow:No assets to save.


INFO:tensorflow:No assets to write.


INFO:tensorflow:No assets to write.


INFO:tensorflow:SavedModel written to: b'trained_models/ssdlite_mobilenet_v2_coco_2018_05_09_exported/saved_model/saved_model.pb'


INFO:tensorflow:SavedModel written to: b'trained_models/ssdlite_mobilenet_v2_coco_2018_05_09_exported/saved_model/saved_model.pb'


Great! Will we be able to find Syd in never seen images? Let's jump to the  `3_prediction` notebook to find the answer.